In [115]:
pip install scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [116]:
pip install sympy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [144]:
from sympy import combinatorics
from sympy.combinatorics import Permutation, PermutationGroup
from sympy.combinatorics import Polyhedron
from sympy.functions.combinatorial.numbers import stirling, bell
from math import factorial, floor
from itertools import permutations
from scipy.stats import chisquare
from numpy.polynomial import Polynomial
import random
import numpy as np
import sympy as sym

## Records count : expected and observable

In [118]:
def count_records_exp(k, n): 
    return stirling(n,k, kind=1) / factorial(n)

In [119]:
def gen_selection_for_permutation_group(N, n): 
    group = [Permutation(x) for x in permutations(range(n))]
    size = factorial(n)
    if N == size: 
        return group
    elif N < size: 
        return random.sample(group, N)
    else: 
        raise Exception('Incorrect selection size, N = {} should be less n = {}'.format(N, size))

In [120]:
def count_records(permutation: Permutation): 
    maximum = -1
    records = 0 
    for index, elem in enumerate(permutation.list()): 
         if elem > maximum: 
            records += 1
            maximum = elem    
    return records

In [122]:
def analyze_records(selection, n): 
    selection_size = len(selection)
    records_counts_ob, records_counts_exp = [0] * n, [0] * n
    
    length = factorial(n)
    for item in selection: 
        records_counts_ob[count_records(item) - 1] += 1
    for i in range(n): 
        records_counts_exp[i] = count_records_exp(i + 1, n) 
        
    (ob, exp) = np.array([float(i / selection_size) for i in records_counts_ob]), np.array([float(i) for i in records_counts_exp])
        
    return chisquare(f_obs=ob, f_exp=exp)

## Shortest cycle length : expected and observable

In [202]:
def expindex(x, M):
    res = x
    for i in range(2, M + 1):
        res += (x**i) / i
    return -res

def nthcoeff(f, x, n):
    res = sym.diff(f, x, n)
    return res.subs(x, 0) / factorial(n)

def get_polynom_for_cycle_length(M, n): 
    x = sym.Symbol('x')
    f = 1 - sym.exp(-x) if  M == 0 else sym.exp(expindex(x, M)) - sym.exp(expindex(x, M + 1))
    return nthcoeff(f / (1 - x), x, n)

def short_cycle_length(permutation):
    return min(permutation.cycle_structure.keys())

In [203]:
def analyze_cycle_length(selection, n): 
    selection_size = len(selection)
    short_cycle_length_ob, short_cycle_length_exp = [0] * n, [0] * n

    length = factorial(n)
    for item in selection: 
        short_cycle_length_ob[short_cycle_length(item) - 1] += 1
    for i in range(floor(n / 2) + 1): 
        short_cycle_length_exp[i] = get_polynom_for_cycle_length(i, n) 
    short_cycle_length_exp[n-1] = get_polynom_for_cycle_length(n-1, n)
        
    (ob, exp) = np.array([float(i / selection_size) for i in short_cycle_length_ob]), np.array([float(i) for i in short_cycle_length_exp])
    ob = np.trim_zeros(np.sort(ob))
    exp = np.trim_zeros(np.sort(exp))
    
    return chisquare(f_obs=ob, f_exp=exp)

In [204]:
selection = gen_selection_for_permutation_group(2_000, 10)
analyze_cycle_length(selection, 10)

Power_divergenceResult(statistic=0.002757623530555431, pvalue=0.999999978779357)